# PySpark ETL with UDF - Local Windows Version

**Environment:** Local Windows with PySpark

**Approach:** Simple UDF with workarounds for Windows compatibility

## Setup: Initialize Spark Session

Configure Spark for local Windows environment

In [1]:
import os
import sys

# Set Python executable for PySpark
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark.sql import SparkSession

# Create Spark session with Windows-friendly config
spark = SparkSession.builder \
    .appName("LocalETL") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

print("✅ Spark Session Created")
print(f"Spark Version: {spark.version}")

✅ Spark Session Created
Spark Version: 3.5.1


## 1. Create Sample Data

In [ ]:
from pyspark.sql import Row
from datetime import date

# Claims data
claims_data = [
    Row(claim_id="CL_001", policyholder_id="PH1", claim_amount=5000, claim_date=date(2024, 1, 15), region="North"),
    Row(claim_id="CL_002", policyholder_id="PH2", claim_amount=3000, claim_date=date(2024, 2, 20), region="South"),
    Row(claim_id="RX_001", policyholder_id="PH3", claim_amount=7000, claim_date=date(2024, 3, 10), region="East"),
    Row(claim_id="CL_003", policyholder_id="PH1", claim_amount=2000, claim_date=date(2024, 4, 5), region="West"),
    Row(claim_id="RX_002", policyholder_id="PH4", claim_amount=4500, claim_date=date(2024, 5, 12), region="North"),
    Row(claim_id="CL_004", policyholder_id="PH2", claim_amount=6000, claim_date=date(2024, 6, 18), region="South"),
]

# Policyholders data
policyholders_data = [
    Row(policyholder_id="PH1", policyholder_name="Alice Johnson"),
    Row(policyholder_id="PH2", policyholder_name="Bob Smith"),
    Row(policyholder_id="PH3", policyholder_name="Charlie Brown"),
    Row(policyholder_id="PH4", policyholder_name="Diana Prince"),
]

# Create DataFrames
claims_df = spark.createDataFrame(claims_data)
policyholders_df = spark.createDataFrame(policyholders_data)

print("✅ Sample data created")
claims_df.show()
policyholders_df.show()

## 2. Define and Register UDF

In [ ]:
import requests
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define the hash function
def get_hash(claim_id):
    """
    Fetches MD4 hash for a claim_id from external API.
    """
    if not claim_id:
        return ""
        
    try:
        url = f"https://api.hashify.net/hash/md4/hex?value={claim_id}"
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            return response.json().get("Digest", "")
        return ""
    except Exception as e:
        print(f"Error: {e}")
        return ""

# Register as UDF
get_hash_udf = udf(get_hash, StringType())

print("✅ UDF registered")

## 3. Extract and Transform Data

In [ ]:
from pyspark.sql.functions import col, when, split, date_format

print("🔄 Starting ETL transformation...\n")

# Step 1: Join Claims and Policyholders
print("📊 Joining claims and policyholders...")
joined_df = claims_df.join(
    policyholders_df, 
    "policyholder_id", 
    "left"
)

# Step 2: Apply UDF to add hash_id column
print("📡 Fetching hashes using UDF...")
joined_with_hashes_df = joined_df.withColumn("hash_id", get_hash_udf(col("claim_id")))

print("\n📋 DataFrame with Hashes:")
joined_with_hashes_df.select("claim_id", "policyholder_name", "hash_id").show(truncate=False)

# Step 3: Apply business transformations
print("\n🔧 Applying business transformations...")
final_df = joined_with_hashes_df.withColumn(
    "claim_type",
    when(col("claim_id").like("CL%"), "Coinsurance")
    .when(col("claim_id").like("RX%"), "Reinsurance")
    .otherwise("Unknown")
).withColumn(
    "claim_priority",
    when(col("claim_amount") > 4000, "Urgent")
    .otherwise("Normal")
).withColumn(
    "claim_period",
    date_format(col("claim_date"), "yyyy-MM")
).withColumn(
    "source_system_id",
    split(col("claim_id"), "_").getItem(1)
)

# Select final columns in specific order
final_df = final_df.select(
    "claim_id",
    "policyholder_name",
    "region",
    "claim_type",
    "claim_priority",
    "claim_amount",
    "claim_period",
    "source_system_id",
    "hash_id"
)

print("\n📊 Final Transformed DataFrame:")
final_df.show(truncate=False)

print("\n✅ Transformation complete!")

## 4. Load - Save Results (Windows-Friendly Approach)

Using pandas to avoid Windows worker crashes

In [ ]:
import pandas as pd

# Define output path (local Windows path)
output_file = r"C:\Users\sivan\Learning\Code\swissre\processed_claims_local.csv"

print(f"💾 Saving to {output_file}...")

# Collect to driver and convert to pandas
pandas_df = final_df.toPandas()

# Write using pandas (more reliable on Windows)
pandas_df.to_csv(output_file, index=False)

print("✅ File saved successfully!")

# Verify
print("\n🔍 Verifying saved file...")
verified_df = pd.read_csv(output_file)
print(f"✅ Read {len(verified_df)} rows from saved file")
print(verified_df.head())

## Alternative: Display Results Without Saving

If you just want to see results without file I/O

In [ ]:
# Show all results
print("📊 Complete Results:")
final_df.show(100, truncate=False)

# Or convert to pandas for better display in Jupyter
display(final_df.toPandas())

## Cleanup: Stop Spark Session

In [ ]:
# Stop Spark when done
spark.stop()
print("✅ Spark session stopped")

## Summary

### ✅ What We Did

1. **Created Spark session** with local Windows configuration
2. **Defined simple UDF** for hash fetching
3. **Applied transformations** using standard PySpark operations
4. **Saved results** using pandas (to avoid Windows worker crashes)

### 🪟 Windows-Specific Workarounds

- ✅ Set `PYSPARK_PYTHON` environment variable
- ✅ Use `toPandas()` + pandas `to_csv()` for saving
- ✅ Reduced shuffle partitions for small dataset
- ✅ Local file paths with `r"C:\..."` format

### 🚀 Running This Notebook

**Prerequisites:**
```bash
pip install pyspark pandas requests jupyter
```

**Run:**
```bash
jupyter notebook
```

Then open this notebook and run all cells!